## **Preparação de ambiente**

In [147]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from matplotlib.colors import LinearSegmentedColormap
from PIL import Image
from wordcloud import WordCloud, ImageColorGenerator

## **Funções auxiliares**

In [148]:
def tabula(atributo,total):
    tabela = atributo.value_counts()
    tabela_frequencia = pd.DataFrame({tabela.index.name: tabela.index})
    tabela_frequencia['F.A.'] = list(tabela)
    tabela_frequencia['F.R.'] = np.nan
    for registro in tabela_frequencia['F.R.']:
        tabela_frequencia['F.R.'] = (tabela_frequencia['F.A.']/total).round(2)
    return tabela_frequencia

def conta_registros(categoria):
  return categoria.sum()

def tabula_categorias(categorias, lista_frequencia, total):
    tabela_frequencia = pd.DataFrame({'Categoria': list(categorias)})
    tabela_frequencia['F.A.'] = lista_frequencia
    tabela_frequencia['F.R.'] = np.nan
    for registro in tabela_frequencia['F.R.']:
        tabela_frequencia['F.R.'] = (tabela_frequencia['F.A.']/total).round(2)
    return tabela_frequencia

def agrupa_frequencias(subcategorias, tabela_frequencia, absoluta):
    fa_categoria = 0  
    for registro in subcategorias:
        fa_categoria +=  tabela_frequencia.at[registro, absoluta]        
    return fa_categoria

def consolida_vies(geral, positivos, negativos, coluna, linha):
    consolidado = geral
    consolidado.rename(columns={'F.A.': 'Frequência absoluta (ni)', 'F.R.': 'Frequência relativa (fi)'}, inplace = True)
    positivos.rename(columns = {'F.A.': 'Positivos (ni)', 'F.R.': 'Positivos (fi)'}, inplace = True)
    negativos.rename(columns = {'F.A.': 'Negativos (ni)', 'F.R.': 'Negativos (fi)'}, inplace = True)
    consolidado = pd.merge(consolidado, positivos, on = coluna, how = 'inner') 
    consolidado = pd.merge(consolidado, negativos, on = coluna, how = 'inner') 
    valores = consolidado.iloc[:, 1:]
    totais = valores.sum(axis = 0)
    consolidado.loc[linha, coluna] = ('Total')
    consolidado.iloc[linha, 1:] = totais
    return consolidado

def tabula_macro_categorias(macro_categorias, absoluta, relativa):    
    agrupamento_itens = [4,5] # subcategorias da categoria "1.2. Agrupamento/Distinção de Itens"
    macro_categorias.at[3, absoluta] = agrupa_frequencias(agrupamento_itens, macro_categorias, absoluta)
    conducao = [2,3,6,7] # subcategorias da categoria "1. Condução"
    macro_categorias.at[1, absoluta] = agrupa_frequencias(conducao, macro_categorias, absoluta)
    brevidade = [10,11] # subcategorias da categoria "2.1. Brevidade"
    macro_categorias.at[9, absoluta] = agrupa_frequencias(brevidade, macro_categorias, absoluta)
    carga = [9,12] # subcategorias da categoria "2. Carga de Trabalho"
    macro_categorias.at[8, absoluta] = agrupa_frequencias(carga, macro_categorias, absoluta)
    controle = [14,15] # subcategorias da categoria "3. Controle Explícito"
    macro_categorias.at[13, absoluta] = agrupa_frequencias(controle, macro_categorias, absoluta)
    adaptabilidade = [17,18] # subcategorias da categoria "4. Adaptabilidade"
    macro_categorias.at[16, absoluta] = agrupa_frequencias(adaptabilidade, macro_categorias, absoluta)
    erros = [20,21,22] # subcategorias da categoria "5. Gestão de Erros"
    macro_categorias.at[19, absoluta] = agrupa_frequencias(erros, macro_categorias, absoluta)
    criterios_ergonomicos_usabilidade = [1,8,13,16,19,23,24,25] # macrocategorias dos "Critérios Ergonômicos de Usabilidade"
    macro_categorias.at[0, absoluta] = agrupa_frequencias(criterios_ergonomicos_usabilidade, macro_categorias, absoluta)
    qualidade_pragmatica = [28,29,30,31,32,33,34] # subcategorias da categoria "1. Qualidade Pragmática"
    macro_categorias.at[27, absoluta] = agrupa_frequencias(qualidade_pragmatica, macro_categorias, absoluta)
    identificacao = [37,38,39,40,41,42,43] # subcategorias da categoria "2.1. Identificação"
    macro_categorias.at[36, absoluta] = agrupa_frequencias(identificacao, macro_categorias, absoluta)
    estimulacao = [45,46,47,48,49,50,51] # subcategorias da categoria "2.2. Estimulação"
    macro_categorias.at[44, absoluta] = agrupa_frequencias(estimulacao, macro_categorias, absoluta)
    qualidade_hedonica = [36,44] # subcategorias da categoria "2. Qualidade Hedônica"
    macro_categorias.at[35, absoluta] = agrupa_frequencias(qualidade_hedonica, macro_categorias, absoluta)
    construtos_avaliativos = [53,54] # subcategorias da categoria "3. Construtos Avaliativos"
    macro_categorias.at[52, absoluta] = agrupa_frequencias(construtos_avaliativos, macro_categorias, absoluta)
    dimensoes_ux = [27,35,32] # macrocategorias das "Dimensões principais da user experience (UX)
    macro_categorias.at[26, absoluta] = agrupa_frequencias(dimensoes_ux, macro_categorias, absoluta)
    principios_ihd = [56,57,58] # macrocategorias dos "Princípios básicos da Interação Humano-Dados (IHD)"
    macro_categorias.at[55, absoluta] = agrupa_frequencias(principios_ihd, macro_categorias, absoluta)
    # totaliza as questões/problemas identificados a partir dos excertos/registos extraidos do teste de usabilidade
    metodos = [0,26,55]
    total_problemas = agrupa_frequencias(metodos, macro_categorias, absoluta)
    for registro in macro_categorias:
        macro_categorias[relativa] =  (macro_categorias[absoluta]/total_problemas).round(2)
    return total_problemas

def cria_esqueleto_arvore():
    colunas_arvore = ['Macrocategoria', 'ni', 'fi', 'Ni', 'Fi', 'Categoria','C (ni)', 'C (fi)', 'C (Ni)', 'C (Fi)', 'Subcategoria', 'S (ni)',
                      'S (fi)', 'S (Ni)', 'S (Fi)', 'Subtipo', 'Sb (ni)', 'Sb (fi)']
    arvore_indices = range(44)
    arvore_categorias = pd.DataFrame(columns = colunas_arvore, index = arvore_indices)
    arvore_categorias.loc[0:17,'Macrocategoria'] = ('Critérios Ergonômicos de Usabilidade')
    arvore_categorias.loc[0:4,'Categoria'] = ('1. Condução')
    arvore_categorias.loc[0, 'Subcategoria'] = ('1.1. Presteza')
    arvore_categorias.loc[1:2,'Subcategoria'] = ('1.2. Agrupamento/Distinção de Itens')
    arvore_categorias.loc[1,'Subtipo'] = ('1.2.1 Agrupamento de Itens - Localização')
    arvore_categorias.loc[2,'Subtipo'] = ('1.2.2 Agrupamento de Itens - Formato')
    arvore_categorias.loc[3,'Subcategoria'] = ('1.3. Feedback Imediato')
    arvore_categorias.loc[4,'Subcategoria'] = ('1.4. Legibilidade')
    arvore_categorias.loc[5:7,'Categoria'] = ('2. Carga de Trabalho')
    arvore_categorias.loc[5:6,'Subcategoria'] =('2.1. Brevidade')
    arvore_categorias.loc[5,'Subtipo'] =('2.1.1 Brevidade - Concisão')
    arvore_categorias.loc[6,'Subtipo'] =('2.1.1 Brevidade - Ações Mínimas')
    arvore_categorias.loc[7,'Subcategoria'] =('2.2. Densidade Informacional')
    arvore_categorias.loc[8:9,'Categoria'] = ('3. Controle Explícito')
    arvore_categorias.loc[8,'Subcategoria'] =('3.1. Ações Explícitas')
    arvore_categorias.loc[9,'Subcategoria'] =('3.2. Controle do Usuário')
    arvore_categorias.loc[10:11,'Categoria'] = ('4. Adaptabilidade')
    arvore_categorias.loc[10,'Subcategoria'] =('4.1. Flexibilidade')
    arvore_categorias.loc[11,'Subcategoria'] =('4.2. Experiência do Usuário')
    arvore_categorias.loc[12:14,'Categoria'] = ('5. Gestão de Erros')
    arvore_categorias.loc[12,'Subcategoria'] =('5.1. Proteção contra Erros')
    arvore_categorias.loc[13,'Subcategoria'] =('5.2. Qualidade das Mensagens de Erros')
    arvore_categorias.loc[14,'Subcategoria'] =('5.3. Correção Erros')
    arvore_categorias.loc[15,'Categoria'] = ('6. Coerência/Consistência (Homogeneidade)')
    arvore_categorias.loc[16,'Categoria'] = ('7. Significado dos Códigos e Denominações')
    arvore_categorias.loc[17,'Categoria'] = ('8. Compatibilidade')
    arvore_categorias.loc[18:41,'Macrocategoria'] = ('Dimensões principais da user experience (UX)')
    arvore_categorias.loc[18:24,'Categoria'] =('1. Qualidade pragmática')
    arvore_categorias.loc[18,'Subcategoria'] =('1.1. Humanidade')
    arvore_categorias.loc[19,'Subcategoria'] =('1.2. Simplicidade')
    arvore_categorias.loc[20,'Subcategoria'] =('1.3. Praticidade')
    arvore_categorias.loc[21,'Subcategoria'] =('1.4. Objetividade')
    arvore_categorias.loc[22,'Subcategoria'] =('1.5. Previsibilidade')
    arvore_categorias.loc[23,'Subcategoria'] =('1.6. Clareza')
    arvore_categorias.loc[24,'Subcategoria'] =('1.7. Controle')
    arvore_categorias.loc[25:38,'Categoria'] =('2. Qualidade hedônica')
    arvore_categorias.loc[25:31,'Subcategoria'] =('2.1. Identificação')
    arvore_categorias.loc[25,'Subtipo'] = ('2.1.1 Integração')
    arvore_categorias.loc[26,'Subtipo'] = ('2.1.2 Profissionalismo')
    arvore_categorias.loc[27,'Subtipo'] = ('2.1.3 Classe')
    arvore_categorias.loc[28,'Subtipo'] = ('2.1.4 Valor')
    arvore_categorias.loc[29,'Subtipo'] = ('2.1.5 Inclusão')
    arvore_categorias.loc[30,'Subtipo'] = ('2.1.6 Socialização')
    arvore_categorias.loc[31,'Subtipo'] = ('2.1.7 Apresentação')
    arvore_categorias.loc[32:38,'Subcategoria'] =('2.2 Estimulação')
    arvore_categorias.loc[32,'Subtipo'] = ('2.2.1 Originalidade')
    arvore_categorias.loc[33,'Subtipo'] = ('2.2.2 Criatividade')
    arvore_categorias.loc[34,'Subtipo'] = ('2.2.3 Coragem')
    arvore_categorias.loc[35,'Subtipo'] = ('2.2.4 Inovação')
    arvore_categorias.loc[36,'Subtipo'] = ('2.2.5 Excitação')
    arvore_categorias.loc[37,'Subtipo'] = ('2.2.6 Desafio')
    arvore_categorias.loc[38,'Subtipo'] = ('2.2.7 Novidade')
    arvore_categorias.loc[39:40,'Categoria'] =('3. Construtos avaliativos')
    arvore_categorias.loc[39,'Subcategoria'] =('3.1. Beleza')
    arvore_categorias.loc[40,'Subcategoria'] =('3.2. Goodness')
    arvore_categorias.loc[41:43,'Macrocategoria'] = ('Princípios básicos da Interação Humano-Dados (IHD)')
    arvore_categorias.loc[41,'Categoria'] =('1. Agência')
    arvore_categorias.loc[42,'Categoria'] =('2. Legibilidade')
    arvore_categorias.loc[43,'Categoria'] =('3. Negociabilidade')
    arvore_categorias.loc[44,'Macrocategoria'] = ('Total')
    return arvore_categorias

def aplicar_mapeamento(mapeamento, coluna_destino, arvore, df_original):
    for destino, origem in mapeamento.items():
        arvore.loc[destino, coluna_destino] = df_original.loc[origem, 'Frequência absoluta (ni)']
    return

def replicar_valores(linha):
    if linha['Sb (ni)'] != 0:
        linha['ni'] = linha['Sb (ni)']
        linha['C (ni)'] = linha['Sb (ni)']
        linha['S (ni)'] = linha['Sb (ni)']
    elif linha['S (ni)'] != 0:
        linha['C (ni)'] = linha['S (ni)']
        linha['ni'] = linha['S (ni)']
    else:
        linha['ni'] = linha['C (ni)']
    return linha

def cria_nuvem(texto, lista_stopwords, mascara, cores):
    # define o gradiente de cores da nuvem de palavras
    gradiente = LinearSegmentedColormap.from_list("mycmap", cores)
    # inicializa a nuvem de palavras
    nuvem = WordCloud(stopwords = lista_stopwords,
                      mask = mascara, # imagem utilizada
                      background_color = 'white', # cor de fundo
                      width = 1500, # largura
                      height = 750, # altura
                      contour_width = 1, # espessura do contorno
                      contour_color = 'purple', # cor do contorno
                      colormap = gradiente
                     )
    # gera a nuvem de palavras a partir do texto
    nuvem.generate(texto)
    # cria visualização
    plt.figure(figsize = (20, 15), facecolor = 'k') # tamanho do gráfico
    plt.imshow(nuvem, interpolation = 'bilinear') # plotagem da nuvem de palavras
    plt.axis('off') # remove as bordas
    plt.show() # mostra a nuvem de palavras

## **Leitura e preparação dos dados**

In [149]:
paineis_covid = pd.read_excel("/home/marisa/Documentos/acadêmico/UFPR/TCC/usabilidade_ux_paineis_covid/dados/experimento_paineis_covid.xlsx", engine = "openpyxl")

### **Criação de subconjuntos de dados**

In [150]:
# cria subconjuntos de registros positivos e negativos
positivos = paineis_covid.query("Viés == 'Positivo'")
negativos = paineis_covid.query("Viés == 'Negativo'")

# cria subconjuntos de registros relacionados ao painel Brasil.io (iniciativa voluntária)
brasil_io = paineis_covid.query("Painel == 'Brasil.io (iniciativa voluntária)'")
brasil_io_positivos = positivos.query("Painel == 'Brasil.io (iniciativa voluntária)'")
brasil_io_negativos = negativos.query("Painel == 'Brasil.io (iniciativa voluntária)'")

# cria subconjuntos de registros relacionados ao painel Coronavírus Brasil (Ministério da Saúde)
coronavirus_brasil = paineis_covid.query("Painel == 'Coronavírus Brasil (Ministério da Saúde)'")
coronavirus_brasil_positivos = positivos.query("Painel == 'Coronavírus Brasil (Ministério da Saúde)'")
coronavirus_brasil_negativos = negativos.query("Painel == 'Coronavírus Brasil (Ministério da Saúde)'")

# cria subconjuntos de registros relacionados ao painel MonitoraCovid-19 (Fiocruz)
monitora_covid = paineis_covid.query("Painel == 'MonitoraCovid-19 (Fiocruz)'")
monitora_covid_positivos = positivos.query("Painel == 'MonitoraCovid-19 (Fiocruz)'")
monitora_covid_negativos = negativos.query("Painel == 'MonitoraCovid-19 (Fiocruz)'")

# cria subconjuntos de registros relacionados ao painel Painel Nacional: Covid-19 (CONASS)
painel_nacional = paineis_covid.query("Painel == 'Painel Nacional: Covid-19 (CONASS)'")
painel_nacional_positivos = positivos.query("Painel == 'Painel Nacional: Covid-19 (CONASS)'")
painel_nacional_negativos = negativos.query("Painel == 'Painel Nacional: Covid-19 (CONASS)'")

## **Tabulação inicial**

In [151]:
# paineis_covid["Usuário"].value_counts()
total_registros = len(paineis_covid.index)
total_positivos = len(positivos.index)
total_negativos = len(negativos.index)
frequencia_vies = tabula(paineis_covid["Viés"], total_registros)
display(frequencia_vies)

,Viés,F.A.,F.R.
0,Negativo,86,0.74
1,Positivo,31,0.26


In [152]:
frequencia_usuarios = tabula(paineis_covid["Usuário"], total_registros)
display(frequencia_usuarios)
frequencia_positivos_usuarios = tabula(positivos["Usuário"], total_positivos)
frequencia_negativos_usuarios = tabula(negativos["Usuário"], total_negativos)

,Usuário,F.A.,F.R.
0,3,57,0.49
1,1,33,0.28
2,2,27,0.23


In [153]:
consolidado_usuarios = consolida_vies(frequencia_usuarios,frequencia_positivos_usuarios,frequencia_negativos_usuarios, 'Usuário', 3)
display(consolidado_usuarios)

/tmp/ipykernel_37501/333990897.py:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Total' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  consolidado.loc[linha, coluna] = ('Total')


,Usuário,Frequência absoluta (ni),Frequência relativa (fi),Positivos (ni),Positivos (fi),Negativos (ni),Negativos (fi)
0,3.0,57.0,0.49,17.0,0.55,40.0,0.47
1,1.0,33.0,0.28,7.0,0.23,26.0,0.30
2,2.0,27.0,0.23,7.0,0.23,20.0,0.23
3,Total,117.0,1.00,31.0,1.01,86.0,1.00


In [154]:
frequencia_paineis = tabula(paineis_covid["Painel"], total_registros)
display(frequencia_paineis)
frequencia_positivos_paineis = tabula(positivos["Painel"], total_positivos)
frequencia_negativos_paineis = tabula(negativos["Painel"], total_negativos)

,Painel,F.A.,F.R.
0,MonitoraCovid-19 (Fiocruz),34,0.29
1,Coronavírus Brasil (Ministério da Saúde),33,0.28
2,Brasil.io (iniciativa voluntária),31,0.26
3,Painel Nacional: Covid-19 (CONASS),19,0.16


In [155]:
consolidado_paineis = consolida_vies(frequencia_paineis,frequencia_positivos_paineis,frequencia_negativos_paineis, 'Painel', 4)
display(consolidado_paineis)

,Painel,Frequência absoluta (ni),Frequência relativa (fi),Positivos (ni),Positivos (fi),Negativos (ni),Negativos (fi)
0,MonitoraCovid-19 (Fiocruz),34.0,0.29,15.0,0.48,19.0,0.22
1,Coronavírus Brasil (Ministério da Saúde),33.0,0.28,2.0,0.06,31.0,0.36
2,Brasil.io (iniciativa voluntária),31.0,0.26,6.0,0.19,25.0,0.29
3,Painel Nacional: Covid-19 (CONASS),19.0,0.16,8.0,0.26,11.0,0.13
4,Total,117.0,0.99,31.0,0.99,86.0,1.00


In [156]:
fontes = paineis_covid.iloc[0:117, 5:8]
frequencias_fontes = []

for atributo in fontes:
    frequencias_fontes.append(conta_registros(fontes[atributo]))

frequencia_fontes_registros = tabula_categorias(fontes, frequencias_fontes, total_registros)
display(frequencia_fontes_registros)

,Categoria,F.A.,F.R.
0,Thinking aloud,82,0.70
1,Registro escrito do teste,79,0.68
2,Observação,32,0.27


## **Tabulação categorias**

In [157]:
categorias = paineis_covid.iloc[0:117, 8:67]
frequencias = []

for atributo in categorias:
    frequencias.append(conta_registros(categorias[atributo]))

frequencia_categorias = tabula_categorias(categorias, frequencias, total_registros)
display(frequencia_categorias)

,Categoria,F.A.,F.R.
0,Critérios Ergonômicos de Usabilidade,110,0.94
1,1. Condução,27,0.23
2,1.1. Presteza,12,0.10
3,1.2. Agrupamento/Distinção de Itens,2,0.02
4,1.2.1 Agrupamento de Itens - localização,0,0.00
5,1.2.2 Agrupamento de Itens - formato,2,0.02
6,1.3. Feedback Imediato,9,0.08
7,1.4. Legibilidade,4,0.03
8,2. Carga de Trabalho,52,0.44
9,2.1. Brevidade,16,0.14


In [158]:
categorias_positivos = positivos.iloc[0:117, 8:67]
frequencias_positivos = []

for atributo in categorias_positivos:
    frequencias_positivos.append(conta_registros(categorias_positivos[atributo]))

frequencia_categorias_positivos = tabula_categorias(categorias_positivos, frequencias_positivos, total_registros)

categorias_negativos = negativos.iloc[0:117, 8:67]
frequencias_negativos = []

for atributo in categorias_negativos:
    frequencias_negativos.append(conta_registros(categorias_negativos[atributo]))

frequencia_categorias_negativos = tabula_categorias(categorias_negativos, frequencias_negativos, total_registros)

In [159]:
macro_categorias = frequencia_categorias
total_problemas = tabula_macro_categorias(macro_categorias, 'F.A.','F.R.')
total_problemas

412

In [160]:
macro_categorias_positivos = frequencia_categorias_positivos
total_problemas_positivos = tabula_macro_categorias(macro_categorias_positivos, 'F.A.', 'F.R.')
total_problemas_positivos

110

In [161]:
macro_categorias_negativos = frequencia_categorias_negativos
total_problemas_negativos = tabula_macro_categorias(macro_categorias_negativos, 'F.A.', 'F.R.')
total_problemas_negativos

302

In [162]:
consolidado_macro_categorias = consolida_vies(macro_categorias,macro_categorias_positivos,macro_categorias_negativos, 'Categoria', 59)
# corrige a totalização levando em consideração a hierarquia de categorias
valores = [total_problemas, 1.0, total_problemas_positivos, 1.0, total_problemas_negativos, 1.0]
consolidado_macro_categorias.iloc[59, 1:] = valores
display(consolidado_macro_categorias)

,Categoria,Frequência absoluta (ni),Frequência relativa (fi),Positivos (ni),Positivos (fi),Negativos (ni),Negativos (fi)
0,Critérios Ergonômicos de Usabilidade,150.0,0.36,31.0,0.28,119.0,0.39
1,1. Condução,27.0,0.07,1.0,0.01,26.0,0.09
2,1.1. Presteza,12.0,0.03,1.0,0.01,11.0,0.04
3,1.2. Agrupamento/Distinção de Itens,2.0,0.00,0.0,0.00,2.0,0.01
4,1.2.1 Agrupamento de Itens - localização,0.0,0.00,0.0,0.00,0.0,0.00
5,1.2.2 Agrupamento de Itens - formato,2.0,0.00,0.0,0.00,2.0,0.01
6,1.3. Feedback Imediato,9.0,0.02,0.0,0.00,9.0,0.03
7,1.4. Legibilidade,4.0,0.01,0.0,0.00,4.0,0.01
8,2. Carga de Trabalho,52.0,0.13,16.0,0.15,36.0,0.12
9,2.1. Brevidade,16.0,0.04,3.0,0.03,13.0,0.04


In [163]:
consolidado_metodos = pd.DataFrame()
indices = [0,26,55,59]
consolidado_metodos = consolidado_macro_categorias.loc[consolidado_macro_categorias.index.isin(indices)]
display(consolidado_metodos)

,Categoria,Frequência absoluta (ni),Frequência relativa (fi),Positivos (ni),Positivos (fi),Negativos (ni),Negativos (fi)
0,Critérios Ergonômicos de Usabilidade,150.0,0.36,31.0,0.28,119.0,0.39
26,Dimensões principais da user experience (UX),177.0,0.43,55.0,0.50,122.0,0.40
55,Princípios básicos da Interação Humano-Dados (...,85.0,0.21,24.0,0.22,61.0,0.20
59,Total,412.0,1.00,110.0,1.00,302.0,1.00


In [171]:
arvore = cria_esqueleto_arvore()           
display(arvore)

,Macrocategoria,ni,fi,Ni,Fi,Categoria,C (ni),C (fi),C (Ni),C (Fi),Subcategoria,S (ni),S (fi),S (Ni),S (Fi),Subtipo,Sb (ni),Sb (fi)
0,Critérios Ergonômicos de Usabilidade,NaN,NaN,NaN,NaN,1. Condução,NaN,NaN,NaN,NaN,1.1. Presteza,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Critérios Ergonômicos de Usabilidade,NaN,NaN,NaN,NaN,1. Condução,NaN,NaN,NaN,NaN,1.2. Agrupamento/Distinção de Itens,NaN,NaN,NaN,NaN,1.2.1 Agrupamento de Itens - Localização,NaN,NaN
2,Critérios Ergonômicos de Usabilidade,NaN,NaN,NaN,NaN,1. Condução,NaN,NaN,NaN,NaN,1.2. Agrupamento/Distinção de Itens,NaN,NaN,NaN,NaN,1.2.2 Agrupamento de Itens - Formato,NaN,NaN
3,Critérios Ergonômicos de Usabilidade,NaN,NaN,NaN,NaN,1. Condução,NaN,NaN,NaN,NaN,1.3. Feedback Imediato,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Critérios Ergonômicos de Usabilidade,NaN,NaN,NaN,NaN,1. Condução,NaN,NaN,NaN,NaN,1.4. Legibilidade,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Critérios Ergonômicos de Usabilidade,NaN,NaN,NaN,NaN,2. Carga de Trabalho,NaN,NaN,NaN,NaN,2.1. Brevidade,NaN,NaN,NaN,NaN,2.1.1 Brevidade - Concisão,NaN,NaN
6,Critérios Ergonômicos de Usabilidade,NaN,NaN,NaN,NaN,2. Carga de Trabalho,NaN,NaN,NaN,NaN,2.1. Brevidade,NaN,NaN,NaN,NaN,2.1.1 Brevidade - Ações Mínimas,NaN,NaN
7,Critérios Ergonômicos de Usabilidade,NaN,NaN,NaN,NaN,2. Carga de Trabalho,NaN,NaN,NaN,NaN,2.2. Densidade Informacional,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Critérios Ergonômicos de Usabilidade,NaN,NaN,NaN,NaN,3. Controle Explícito,NaN,NaN,NaN,NaN,3.1. Ações Explícitas,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Critérios Ergonômicos de Usabilidade,NaN,NaN,NaN,NaN,3. Controle Explícito,NaN,NaN,NaN,NaN,3.2. Controle do Usuário,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [172]:
# dicionário que mapeia índices de destino de subtipos em 'arvore' aos índices de origem em 'consolidado_macro_categorias'
mapeamento_subtipos = {
    1: 4, 2: 5,
    5: 10, 6: 11,
    25: 37, 26: 38, 27: 39, 28: 40, 29: 41, 30: 42, 31: 43,
    32: 45, 33: 46, 34: 47, 35: 48, 36: 49, 37: 50, 38: 51
}

# dicionário que mapeia índices de destino de subcategorias sem subtipos em 'arvore' aos índices de origem em 'consolidado_macro_categorias'
mapeamento_subcategorias = {
    0: 2,
    3: 6, 4: 7,
    7: 12, 8: 14, 9: 15, 10: 17, 11: 18, 12: 20, 13: 21, 14: 22,
    18: 28, 19: 29, 20: 30, 21: 31, 22: 32, 23: 33, 24: 34,
    39: 53, 40: 54
}

# dicionário que mapeia índices de destino de categorias sem subcategorias em 'arvore' aos índices de origem em 'consolidado_macro_categorias'
mapeamento_categorias = {
    15: 23, 16: 24, 17: 25,
    41: 56, 42: 57, 43: 58
}

# popula as colunas de frequência absoluta da árvore a partir dos mapeamentos
aplicar_mapeamento(mapeamento_subtipos, 'Sb (ni)', arvore, consolidado_macro_categorias)
aplicar_mapeamento(mapeamento_subcategorias, 'S (ni)', arvore, consolidado_macro_categorias)
aplicar_mapeamento(mapeamento_categorias, 'C (ni)', arvore, consolidado_macro_categorias)

# lista de colunas nas quais o nulo será substituído por zero
colunas_numericas = ['Sb (ni)', 'Sb (fi)', 'S (ni)', 'S (fi)', 'S (Ni)', 'S (Fi)', 'C (ni)', 'C (fi)', 'C (Ni)', 'C (Fi)',
                     'ni', 'fi', 'Ni', 'Fi']

# trata nulos susbstindo NaN por zero
arvore[colunas_numericas] = arvore[colunas_numericas].fillna(0)
        
# replica valores de frequência para níveis superios da árvore
arvore = arvore.apply(replicar_valores, axis = 1)
    
display(arvore)

,Macrocategoria,ni,fi,Ni,Fi,Categoria,C (ni),C (fi),C (Ni),C (Fi),Subcategoria,S (ni),S (fi),S (Ni),S (Fi),Subtipo,Sb (ni),Sb (fi)
0,Critérios Ergonômicos de Usabilidade,12.0,0,0,0,1. Condução,12.0,0,0,0,1.1. Presteza,12.0,0,0,0,NaN,0.0,0
1,Critérios Ergonômicos de Usabilidade,0.0,0,0,0,1. Condução,0.0,0,0,0,1.2. Agrupamento/Distinção de Itens,0.0,0,0,0,1.2.1 Agrupamento de Itens - Localização,0.0,0
2,Critérios Ergonômicos de Usabilidade,2.0,0,0,0,1. Condução,2.0,0,0,0,1.2. Agrupamento/Distinção de Itens,2.0,0,0,0,1.2.2 Agrupamento de Itens - Formato,2.0,0
3,Critérios Ergonômicos de Usabilidade,9.0,0,0,0,1. Condução,9.0,0,0,0,1.3. Feedback Imediato,9.0,0,0,0,NaN,0.0,0
4,Critérios Ergonômicos de Usabilidade,4.0,0,0,0,1. Condução,4.0,0,0,0,1.4. Legibilidade,4.0,0,0,0,NaN,0.0,0
5,Critérios Ergonômicos de Usabilidade,0.0,0,0,0,2. Carga de Trabalho,0.0,0,0,0,2.1. Brevidade,0.0,0,0,0,2.1.1 Brevidade - Concisão,0.0,0
6,Critérios Ergonômicos de Usabilidade,16.0,0,0,0,2. Carga de Trabalho,16.0,0,0,0,2.1. Brevidade,16.0,0,0,0,2.1.1 Brevidade - Ações Mínimas,16.0,0
7,Critérios Ergonômicos de Usabilidade,36.0,0,0,0,2. Carga de Trabalho,36.0,0,0,0,2.2. Densidade Informacional,36.0,0,0,0,NaN,0.0,0
8,Critérios Ergonômicos de Usabilidade,0.0,0,0,0,3. Controle Explícito,0.0,0,0,0,3.1. Ações Explícitas,0.0,0,0,0,NaN,0.0,0
9,Critérios Ergonômicos de Usabilidade,6.0,0,0,0,3. Controle Explícito,6.0,0,0,0,3.2. Controle do Usuário,6.0,0,0,0,NaN,0.0,0


### **Categorias - Brasil.io (iniciativa voluntária)**

### **Categorias - Coronavírus Brasil (Ministério da Saúde)**

### **Categorias - MonitoraCovid-19 (Fiocruz)**

### **Categorias - Painel Nacional: Covid-19 (CONASS)**



### **Categorias - Coronavírus Brasil (Ministério da Saúde)**

### **Categorias - MonitoraCovid-19 (Fiocruz)**

### **Categorias - Painel Nacional: Covid-19 (CONASS)**

## **Nuvens de palavras**

**Passos iniciais**

In [ ]:
# delimita as palavras dos comentários que devem ser eliminadas da nuvem de palavras
lista_stopwords = ['o','a','os','as','um','uma','uns','umas','de','da','do','das','dos','em','no','na','nos','nas','à','ao',
                   'para','pra','prá','pro','pras','prás','pros','neste','nesta','nestes','nestas','deste','desta','destes',
                   'destas','nesses','nessas','desse','dessa','desses','dessas','nisso','nisto','naquilo','disso','disto',
                   'daquilo','naquele','naquela','naqueles','naquelas','daquele','daquela','daqueles','daquelas','àquele',
                   'àquela','àqueles','àquelas','àquilo','ante','após','até','contra','desde','entre','perante','por','per',
                   'sob','sobre','trás','pelo','pela','pelos','pelas','outro','outra','outros','outras','eu','você','ele',
                   'ela','eles','elas','seu','sua','seus','suas','teu','tua','teus','tuas','dele','dela','deles','delas',
                   'nele','nela','neles','nelas','meu','me','comigo','consigo','contigo','isto','isso','aquilo','este','esta',
                   'estes','estas','esse','essa','esses','essas','aquele','aquela','aqueles','aquelas','todo','toda','todos',
                   'todas','cada','que','muitos','poucos','ser','sendo','é','foi','era','sou','somos','são','era','éramos',
                   'eram','fui','foi','fomos','foram','fora','fôramos','seja','sejamos','sejam','fosse','fôssemos','fossem',
                   'for','formos','forem','serei','será','seremos','serão','seria','seríamos','seriam','estar','está','estão',
                   'tá','tão','estando','estou','estamos','estive','esteve','estivemos','estiveram','estava','estávamos','estavam',
                   'estivera','estivéramos','esteja','estejamos','estejam','estivesse','estivéssemos','estivessem','estiver',
                   'estivermos','estiverem','ir','ía','ia','iria','fui','indo','vir','vai','vou','vindo','ficar','ficou','ficam',
                   'ficando','fiquei','fazer','faz', 'fez','feita', 'feito','e','ou','mas','também','se','nem','ai','aí','aqui',
                   'lá','ali','já','não','mais','menos','muito','pouco','assim','bem','mal','bastante','só','somente',
                   'onde','como', 'quando','quem','tudo','nada','com','sem', 'ID1','obs','.','duas','mil','painel','dados',
                   'primeiro', 'primeiros', 'primeira', 'primeiras', 'depois', 'qual', 'quais', 'qualquer', 'quaisquer'
                   #'dado','painéis','página','informação','informações','site','município','municípios','estado','estados','PR','letalidade',
                   ]


# fazer uma visão com 'dados' e 'painel' e as demais sem

# abre uma imagem e a transforma em um array do numpy
mascara_covid = np.array(Image.open('/home/marisa/Documentos/acadêmico/UFPR/TCC/usabilidade_ux_paineis_covid/imagens/covid_4.png'))

**Nuvens completas**

- Geral - todos os comentários de todos os painéis:

In [ ]:
# define quais serão os textos analisados para formar a nuvem de palavras
comentarios = ' '.join(paineis_covid["Comentário"].tolist())

# define o gradiente de cores da nuvem de palavras
cores = ["#BF0A30", "#002868"]

# gera e exibe a visualização da nuvem de palavras
cria_nuvem(comentarios,lista_stopwords,mascara_covid,cores)

- Brasil.io (iniciativa voluntária) - todos os comentários referentes ao painel:

In [ ]:
comentarios_brasil_io = ' '.join(brasil_io["Comentário"].tolist())
cores = ["#BF0A30", "#002868"]
cria_nuvem(comentarios_brasil_io,lista_stopwords,mascara_covid,cores)

- Coronavírus Brasil (Ministério da Saúde) - todos os comentários referentes ao painel:

In [ ]:
comentarios_coronavirus_brasil = ' '.join(coronavirus_brasil["Comentário"].tolist())
cores = ["#BF0A30", "#002868"]
cria_nuvem(comentarios_coronavirus_brasil,lista_stopwords,mascara_covid,cores)

- MonitoraCovid-19 (Fiocruz) - todos os comentários referentes ao painel:

In [ ]:
comentarios_monitora_covid = ' '.join(monitora_covid["Comentário"].tolist())
cores = ["#BF0A30", "#002868"]
cria_nuvem(comentarios_monitora_covid,lista_stopwords,mascara_covid,cores)

- Painel Nacional: Covid-19 (CONASS) - todos os comentários referentes ao painel:

In [ ]:
comentarios_painel_nacional = ' '.join(painel_nacional["Comentário"].tolist())
cores = ["#BF0A30", "#002868"]
cria_nuvem(comentarios_painel_nacional,lista_stopwords,mascara_covid,cores)

**Nuvens registros positivos**

- Geral - comentários positivos de todos os painéis:

In [ ]:
comentarios_positivos = ' '.join(positivos["Comentário"].tolist())
cores = ["#BF0A30", "#002868"]
cria_nuvem(comentarios_positivos,lista_stopwords,mascara_covid,cores)

- Brasil.io (iniciativa voluntária) - comentários positivos referentes ao painel:

In [ ]:
comentarios_positivos_brasil_io = ' '.join(brasil_io_positivos["Comentário"].tolist())
cores = ["#BF0A30", "#002868"]
cria_nuvem(comentarios_positivos_brasil_io,lista_stopwords,mascara_covid,cores)

- Coronavírus Brasil (Ministério da Saúde) - comentários positivos referentes ao painel:

In [ ]:
comentarios_positivos_coronavirus_brasil = ' '.join(coronavirus_brasil_positivos["Comentário"].tolist())
cores = ["#BF0A30", "#002868"]
cria_nuvem(comentarios_positivos_coronavirus_brasil,lista_stopwords,mascara_covid,cores)

- MonitoraCovid-19 (Fiocruz) - comentários positivos referentes ao painel:

In [ ]:
comentarios_positivos_monitora_covid = ' '.join(monitora_covid_positivos["Comentário"].tolist())
cores = ["#BF0A30", "#002868"]
cria_nuvem(comentarios_positivos_monitora_covid,lista_stopwords,mascara_covid,cores)

- Painel Nacional: Covid-19 (CONASS) - comentários positivos referentes ao painel:

In [ ]:
comentarios_positivos_painel_nacional = ' '.join(painel_nacional_positivos["Comentário"].tolist())
cores = ["#BF0A30", "#002868"]
cria_nuvem(comentarios_positivos_painel_nacional,lista_stopwords,mascara_covid,cores)

**Nuvens registros negativos**

- Geral - comentários negativos de todos os painéis:

In [ ]:
comentarios_negativos = ' '.join(negativos["Comentário"].tolist())
cores = ["#BF0A30", "#002868"]
cria_nuvem(comentarios_negativos,lista_stopwords,mascara_covid,cores)

- Brasil.io (iniciativa voluntária) - comentários negativos referentes ao painel:

In [ ]:
comentarios_negativos_brasil_io = ' '.join(brasil_io_negativos["Comentário"].tolist())
cores = ["#BF0A30", "#002868"]
cria_nuvem(comentarios_negativos_brasil_io,lista_stopwords,mascara_covid,cores)

- Coronavírus Brasil (Ministério da Saúde) - comentários negativos referentes ao painel:

In [ ]:
comentarios_negativos_coronavirus_brasil = ' '.join(coronavirus_brasil_negativos["Comentário"].tolist())
cores = ["#BF0A30", "#002868"]
cria_nuvem(comentarios_negativos_coronavirus_brasil,lista_stopwords,mascara_covid,cores)

- MonitoraCovid-19 (Fiocruz) - comentários negativos referentes ao painel:

In [ ]:
comentarios_negativos_monitora_covid = ' '.join(monitora_covid_negativos["Comentário"].tolist())
cores = ["#BF0A30", "#002868"]
cria_nuvem(comentarios_negativos_monitora_covid,lista_stopwords,mascara_covid,cores)

- Painel Nacional: Covid-19 (CONASS) - comentários negativos referentes ao painel:

In [ ]:
comentarios_negativos_painel_nacional = ' '.join(painel_nacional_negativos["Comentário"].tolist())
cores = ["#BF0A30", "#002868"]
cria_nuvem(comentarios_negativos_painel_nacional,lista_stopwords,mascara_covid,cores)

**Tabulação usuário 1 - Ergonomia/Design**

In [ ]:
usuario_1 = paineis_covid.query("Usuário == 1")
display(usuario_1)

In [ ]:
total_usuario_1 = len(usuario_1.index)
frequencia_paineis_usuario_1 = tabula(usuario_1["Painel"], total_usuario_1)
display(frequencia_paineis_usuario_1)

In [ ]:
frequencia_vies_usuario_1 = tabula(usuario_1["Viés"], total_usuario_1)
display(frequencia_vies_usuario_1)

In [ ]:
categorias_usuario_1 = usuario_1.iloc[0:117, 5:61]
frequencias_usuario_1 = []

for atributo in categorias_usuario_1:
    frequencias_usuario_1.append(conta_registros(categorias_usuario_1[atributo]))

frequencia_categorias_usuario_1 = tabula_categorias(categorias_usuario_1, frequencias_usuario_1, total_usuario_1)
display(frequencia_categorias_usuario_1)

In [ ]:
# define quais serão os textos analisados para formar a nuvem de palavras
comentarios_usuario_1 = ' '.join(usuario_1["Comentário"].tolist())

# define o gradiente de cores da nuvem de palavras
cores_1 = ["#FFA500", "#61088A"]
gradiente_cores_1 = LinearSegmentedColormap.from_list("mycmap", cores_1)

# inicializa a nuvem de palavras
nuvem_1 = WordCloud(stopwords = lista_stopwords,
                  mask = mascara_covid, # imagem utilizada
                  background_color = 'white', # cor de fundo
                  width = 1500, # largura
                  height = 750, # altura
                  contour_width = 1, # espessura do contorno
                  contour_color = 'purple', # cor do contorno
                  colormap = gradiente_cores_1
                 )

# gera a nuvem de palavras a partir do texto dos comentários do usuário 1
nuvem_1.generate(comentarios_usuario_1)

plt.figure(figsize = (20, 15), facecolor = 'k') # tamanho do gráfico
plt.imshow(nuvem_1, interpolation = 'bilinear') # plotagem da nuvem de palavras
plt.axis('off') # remove as bordas
plt.show() # mostra a nuvem de palavras

**Tabulação usuário 2 - Dados**

In [ ]:
usuario_2 = paineis_covid.query("Usuário == 2")
display(usuario_2)

In [ ]:
total_usuario_2 = len(usuario_2.index)
frequencia_paineis_usuario_2 = tabula(usuario_2["Painel"], total_usuario_2)
display(frequencia_paineis_usuario_2)

In [ ]:
frequencia_vies_usuario_2 = tabula(usuario_2["Viés"], total_usuario_2)
display(frequencia_vies_usuario_2)

In [ ]:
categorias_usuario_2 = usuario_2.iloc[0:117, 5:61]
frequencias_usuario_2 = []

for atributo in categorias_usuario_2:
    frequencias_usuario_2.append(conta_registros(categorias_usuario_2[atributo]))

frequencia_categorias_usuario_2 = tabula_categorias(categorias_usuario_2, frequencias_usuario_2, total_usuario_2)
display(frequencia_categorias_usuario_2)

In [ ]:
# define quais serão os textos analisados para formar a nuvem de palavras
comentarios_usuario_2 = ' '.join(usuario_2["Comentário"].tolist())

# define o gradiente de cores da nuvem de palavras
cores_2 = ["#059E14", "#9904B0"]
gradiente_cores_2 = LinearSegmentedColormap.from_list("mycmap", cores_2)

# inicializa a nuvem de palavras
nuvem_2 = WordCloud(stopwords = lista_stopwords,
                  mask = mascara_covid, # imagem utilizada
                  background_color = 'white', # cor de fundo
                  width = 1500, # largura
                  height = 750, # altura
                  contour_width = 1, # espessura do contorno
                  contour_color = 'purple', # cor do contorno
                  colormap = gradiente_cores_2
                 )

# gera a nuvem de palavras a partir do texto dos comentários do usuário 2
nuvem_2.generate(comentarios_usuario_2)

plt.figure(figsize = (20, 15), facecolor = 'k') # tamanho do gráfico
plt.imshow(nuvem_2, interpolation = 'bilinear') # plotagem da nuvem de palavras
plt.axis('off') # remove as bordas
plt.show() # mostra a nuvem de palavras

**Tabulação usuário 3 - Saúde**

In [ ]:
usuario_3 = paineis_covid.query("Usuário == 3")
display(usuario_3)

In [ ]:
total_usuario_3 = len(usuario_3.index)
frequencia_paineis_usuario_3 = tabula(usuario_3["Painel"], total_usuario_3)
display(frequencia_paineis_usuario_3)

In [ ]:
frequencia_vies_usuario_3 = tabula(usuario_3["Viés"], total_usuario_3)
display(frequencia_vies_usuario_3)

In [ ]:
categorias_usuario_3 = usuario_3.iloc[0:117, 5:61]
frequencias_usuario_3 = []

for atributo in categorias_usuario_3:
    frequencias_usuario_3.append(conta_registros(categorias_usuario_3[atributo]))

frequencia_categorias_usuario_3 = tabula_categorias(categorias_usuario_3, frequencias_usuario_3, total_usuario_3)
display(frequencia_categorias_usuario_3)

In [ ]:
# define quais serão os textos analisados para formar a nuvem de palavras
comentarios_usuario_3 = ' '.join(usuario_3["Comentário"].tolist())

# define o gradiente de cores da nuvem de palavras
cores_3 = ["#000AC4", "#DB047E"]
gradiente_cores_3 = LinearSegmentedColormap.from_list("mycmap", cores_3)

# inicializa a nuvem de palavras
nuvem_3 = WordCloud(stopwords = lista_stopwords,
                  mask = mascara_covid, # imagem utilizada
                  background_color = 'white', # cor de fundo
                  width = 1500, # largura
                  height = 750, # altura
                  contour_width = 1, # espessura do contorno
                  contour_color = 'purple', # cor do contorno
                  colormap = gradiente_cores_3
                 )

# gera a nuvem de palavras a partir do texto dos comentários do usuário 3
nuvem_3.generate(comentarios_usuario_3)

plt.figure(figsize = (20, 15), facecolor = 'k') # tamanho do gráfico
plt.imshow(nuvem_3, interpolation = 'bilinear') # plotagem da nuvem de palavras
plt.axis('off') # remove as bordas
plt.show() # mostra a nuvem de palavras